In [ ]:
#We need to built two model 
#We will first build a logistic regression and Support vector machine second

In [1]:
#import libs
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import pickle

In [2]:
#work on spliting data

#open x abd y df's
with open('X_res', 'rb') as x:
    X_res = pickle.load(x)

In [3]:
with open('y_res', 'rb') as y:
    y_res = pickle.load(y)

In [4]:
#split x and y data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_res,
                                  y_res,
                                  test_size=0.2,
                                  random_state=1)



In [5]:
#since we need to use Auc as metric evaluation
score = 'roc_auc'

In [6]:
#logistic regression without feature selection
classifier = LogisticRegression(random_state = 0,max_iter=10000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [7]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, \
    recall_score
print('AUC: ', roc_auc_score(y_test, y_pred))
print ("Accuracy : ", accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))

AUC:  0.72239926445074
Accuracy :  0.7221288105855691
Precision:  0.7073887489504618
Recall:  0.7472283813747228


In [9]:
#Setup recursive feature reduction w/ cross validation
clf2 = RFECV(LogisticRegression(max_iter=10000),
      scoring = score,
      n_jobs = -1,
      cv = 3,
      step = 5)

In [10]:
clf2.fit(X_train, y_train)

RFECV(cv=3, estimator=LogisticRegression(max_iter=10000), n_jobs=-1,
      scoring='roc_auc', step=5)

In [11]:
#Generate predicted probabilites
clf2_probs = clf2.predict_proba(X_test)
print('AUC: ', roc_auc_score(y_test, clf2_probs[:,1]))
print('Accuracy: ', clf2.score(X_test, y_test))

AUC:  0.794756485603618
Accuracy:  0.7218363915490899


In [12]:
#Pickle logistic regression
with open('clf2', 'wb') as c:
    pickle.dump(clf2, c, pickle.HIGHEST_PROTOCOL)

In [ ]:
import sys
!{sys.executable} -m pip install xgboost

In [23]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [24]:
param = {
'objective': 'multi:softprob',
'eta': 0.4,
'max_depth': 3,
'gamma': 1.0,
'num_class': 3 }

In [25]:
model = xgb.train(param, dtrain,100)

In [30]:
y_pred = model.predict(dtest)
predicts = np.asarray([np.argmax(val) for val in y_pred])
y_pred

array([[9.9424970e-01, 5.7478831e-03, 2.4058534e-06],
       [5.4970793e-03, 9.9450016e-01, 2.7917201e-06],
       [7.4510253e-04, 9.9925405e-01, 8.8578895e-07],
       ...,
       [5.5116928e-01, 4.4881216e-01, 1.8611692e-05],
       [9.4459903e-01, 5.5393264e-02, 7.6806218e-06],
       [9.6736139e-01, 3.2632697e-02, 5.9070171e-06]], dtype=float32)

In [27]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, predicts)
print('Accuracy = {}'.format(acc))
print('AUC: ', roc_auc_score(y_test, predicts))

Accuracy = 0.918122669785803
AUC:  0.9175363632301481


In [29]:
#Pickle xgboost model
with open('model', 'wb') as c:
    pickle.dump(model, c, pickle.HIGHEST_PROTOCOL)